In [11]:
import datetime
import subprocess
import os

import pandas as pd
from graphreduce.node import GraphReduceNode, DynamicNode
from graphreduce.graph_reduce import GraphReduce
from graphreduce.enum import ComputeLayerEnum, PeriodUnit

In [2]:
# We will focus on doing filters
# to highlight point in time correctness

In [3]:
files = [
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv',
]

for f in os.listdir(os.getcwd()):
    for _f in files:
        _fname = _f.split('/')[-1]
        if _fname == f or _fname in f:
            os.remove(f)

for f in files:
  subprocess.run(['wget', f])

--2024-08-30 11:58:04--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35 [text/plain]
Saving to: ‘cust.csv’

     0K                                                       100% 1.52M=0s

2024-08-30 11:58:04 (1.52 MB/s) - ‘cust.csv’ saved [35/35]

--2024-08-30 11:58:04--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191 [text/plain]
Savin

In [12]:
# Need unique prefixes for all nodes
# so when columns are merged we know
# where they originate from.
prefixes = {
    'cust.csv' : {'prefix':'cu'},
    'orders.csv':{'prefix':'ord'}
}

In [13]:
# create graph reduce nodes
gr_nodes = {
    f.split('/')[-1]: DynamicNode(
        fpath=f,
        fmt='csv',
        pk='id',
        prefix=prefixes[f]['prefix'],
        date_key=None,
        compute_layer=GraphReduceComputeLayerEnum.pandas,
        compute_period_val=730,
        compute_period_unit=PeriodUnit.day,
    )
    for f in prefixes.keys()
}

2024-08-30 12:15:28 [warning  ] no `date_key` set for <GraphReduceNode: fpath=cust.csv fmt=csv>
2024-08-30 12:15:28 [warning  ] no `date_key` set for <GraphReduceNode: fpath=orders.csv fmt=csv>


In [14]:
# Instantiate a single node with a specific date

In [15]:
!cat orders.csv

id,customer_id,ts,amount
1,1,2023-05-12,10
2,1,2023-06-01,12
3,2,2023-01-01,13
4,2,2022-08-05,150
5,3,2023-06-01,220
6,1,2023-09-02,1200
7,2,2023-10-15,47
8,4,2024-01-01,42
9,4,2024-02-01,42


In [20]:
order_node = DynamicNode(
    fpath='./orders.csv',
    fmt='csv',
    pk='id',
    prefix='ord',
    date_key='ts',
    compute_layer=ComputeLayerEnum.pandas,
    compute_period_val=180,
    compute_period_unit=PeriodUnit.day,
    cut_date=datetime.datetime(2023,10,1),
    label_period_val=30,
    label_period_unit=PeriodUnit.day
)

In [23]:
order_node.do_data()

In [24]:
print(len(order_node.df))

9


In [26]:
print(len(order_node.prep_for_features()))

4


In [27]:
print(len(order_node.prep_for_labels()))

1


In [30]:
# instantiate the top-level graph reduce object like so:

gr = GraphReduce(
    name='starter_graph',
    parent_node=gr_nodes['cust.csv'],
    fmt='csv',
    cut_date=datetime.datetime(2023,9,1),
    compute_layer=GraphReduceComputeLayerEnum.pandas,
    auto_features=True,
    auto_feature_hops_front=1,
    auto_feature_hops_back=2,
    label_node=gr_nodes['orders.csv'],
    label_operation='count',
    label_field='id',
    label_period_val=60,
    label_period_unit=PeriodUnit.day
)